In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/concrete_ai')
cwd = os.getcwd()
print('current directory : ', cwd)

Mounted at /content/drive
current directory :  /content/drive/MyDrive/concrete_ai


In [ ]:
import os
import data
import numpy as np
import pandas as pd
from utils import utils
import tensorflow as tf
import printing_callback as callback
from keras.layers.core import Dense
from keras.models import Sequential
from keras.layers import LayerNormalization, Dropout
from tensorflow import keras
from tensorflow.keras.layers.experimental import RandomFourierFeatures

In [ ]:
INPUT_TITLE = 'amj_data4'
OUTPUT_TITLE = 'amj_data4_7days'

In [ ]:
train, test, validation = data.getFinalData('data_files/' + INPUT_TITLE + '.xlsx')
#train = train[train[:,5] == 30]
#test = test[test[:,5] == 30]
#validation = validation[validation[:,5] == 30]
train, test, validation = data.prepareMultipleData(train, test, validation, [13, 14, 16])

#train[:,13], tr1, tr2 = minMaxNormalizeOutput(train[:,13])
#test[:,13], te1, te2 = minMaxNormalizeOutput(test[:,13])
#validation[:,13], va1, va2 = minMaxNormalizeOutput(validation[:,13])

tmp1 = train.shape[1] - 1
tmp2 = train.shape[1]
 
xr = train[:,0:tmp1]
yr = train[:,tmp1:tmp2]
xt = test[:,0:tmp1]
yt = test[:,tmp1:tmp2]
xv = validation[:,0:tmp1]
yv = validation[:,tmp1:tmp2]
 
utils.exceptionIfNan(train)
utils.exceptionIfNan(test)
utils.exceptionIfNan(validation)
 
print('data ready with train:', train.shape, 'and test:', test.shape,
      'and validation:', validation.shape)

loading the data ...
data loaded with size:  (914, 17) 


there is Nan in data


shuffling the data ...
data shuffled


dividing the data ...
training data ready with size: (640, 17)
test data ready with size: (138, 17)
validation data ready with size: (136, 17)
data ready with train: (633, 14) and test: (138, 14) and validation: (135, 14)


In [ ]:
print(xr[1])
print(yr[1])

[2.000e+00 1.000e+00 2.000e+01 5.410e+01 4.300e+01 3.000e+01 3.750e+02
 5.100e-01 1.900e+02 0.000e+00 1.885e+03 6.800e+02 1.205e+03]
[31.1]


In [ ]:
data.saveData(train, 'train2.xlsx')

In [ ]:
model = utils.newSeqentialModel(xr.shape[1], yr.shape[1])
#from keras.models import Model, Sequential
#from keras.layers.core import Dense, Dropout
#from keras.layers import LayerNormalization

#model = Sequential()
#model.add(LayerNormalization(input_dim=xr.shape[1]))
#model.add(Dropout(0.3))
#model.add(Dense(20, activation='relu'))
#model.add(Dropout(0.3))
#model.add(Dense(10, activation='relu'))
#model.add(Dropout(0.3))
#model.add(Dense(5, activation='relu'))
#model.add(Dense(yr.shape[1]))

In [ ]:
print('loading model weights ...')
output_dir = os.path.join(os.getcwd(), "saved_wights")
try:
    model.load_weights(filepath=os.path.join(output_dir, OUTPUT_TITLE + ".h5"))
    print('model weights loaded')
except OSError:
    print('no previous weights found')
except ValueError:
    print('previous weights are different from current') 

loading model weights ...
no previous weights found


In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', 
metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

In [ ]:
print('Trainning model using ANN\n')
model.fit(xr, yr, epochs=20000, batch_size=32, validation_data=(xv, yv), 
verbose=0, callbacks=[callback.LossAndErrorPrintingCallback(),
                      tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2000),])
print('\nmodel trained')

Trainning model using ANN

epoch 0 : loss :  900.35 , MPE :   98.90. validation loss :  884.70 , MPE :   97.92
epoch 100 : loss :   28.39 , MPE :   15.48. validation loss :   28.34 , MPE :   15.26
epoch 200 : loss :   28.39 , MPE :   15.46. validation loss :   28.34 , MPE :   15.25
epoch 300 : loss :   28.40 , MPE :   15.48. validation loss :   28.34 , MPE :   15.24
epoch 400 : loss :   28.40 , MPE :   15.50. validation loss :   28.33 , MPE :   15.26
epoch 500 : loss :   28.40 , MPE :   15.48. validation loss :   28.33 , MPE :   15.27
epoch 600 : loss :   28.40 , MPE :   15.46. validation loss :   28.34 , MPE :   15.25
epoch 700 : loss :   28.40 , MPE :   15.51. validation loss :   28.33 , MPE :   15.29
epoch 800 : loss :   28.45 , MPE :   15.49. validation loss :   28.34 , MPE :   15.30
epoch 900 : loss :   28.40 , MPE :   15.48. validation loss :   28.34 , MPE :   15.25
epoch 1000 : loss :   28.39 , MPE :   15.48. validation loss :   28.33 , MPE :   15.27
epoch 1100 : loss :   28.41 

In [ ]:
_, accuracy_test = model.evaluate(xt, yt)
print('\nAccuracy on test data: %.2f' % (accuracy_test))

5/5 [==============================] - 0s 4ms/step - loss: 37.9008 - mean_absolute_percentage_error: 16.4772

Accuracy on test data: 16.48


In [ ]:
test_predictions = np.around(model.predict(xt), 1)
 
for i in range(20):
    print(' test: predicted:', test_predictions[i], 'real data:', yt[i])

 test: predicted: [26.8] real data: [18.3]
 test: predicted: [32.] real data: [35.5]
 test: predicted: [29.8] real data: [36.1]
 test: predicted: [29.] real data: [35.1]
 test: predicted: [30.] real data: [38.8]
 test: predicted: [32.] real data: [30.5]
 test: predicted: [36.3] real data: [27.3]
 test: predicted: [29.1] real data: [32.6]
 test: predicted: [35.7] real data: [28.6]
 test: predicted: [29.6] real data: [26.3]
 test: predicted: [28.4] real data: [28.6]
 test: predicted: [27.7] real data: [32.]
 test: predicted: [30.] real data: [34.6]
 test: predicted: [30.] real data: [41.2]
 test: predicted: [23.9] real data: [33.9]
 test: predicted: [27.9] real data: [27.7]
 test: predicted: [33.] real data: [37.9]
 test: predicted: [35.3] real data: [32.8]
 test: predicted: [30.1] real data: [25.]
 test: predicted: [36.] real data: [27.]


**Testing models against lab results**

In [ ]:
# For 7 days
custom = np.array([[1, 1, 20, 26, 38, 25, 350, 0.5, 175, 0, 1875, 725, 1150],
                   [1, 1, 20, 26, 43, 30, 375, 0.47, 175, 3, 1850, 700, 1150]])
print('testing custom data for 7 days...')
customResult = np.around(model.predict(custom), 1)
print('Grade 25 :')
print('expected result : 21.7')
print('predicted result :', customResult[0,0])
print('Grade 30 :')
print('expected result : 29.9')
print('predicted result :', customResult[1,0])

testing custom data for 7 days...
Grade 25 :
expected result : 21.7
predicted result : 24.3
Grade 30 :
expected result : 29.9
predicted result : 27.0


In [ ]:
# For 28 days
custom = np.array([[1, 1, 20, 26, 38, 25, 350, 0.5, 175, 0, 1875, 725, 1150],
                   [1, 1, 20, 26, 43, 30, 375, 0.47, 175, 3, 1850, 700, 1150]])
print('testing custom data for 28 days...')
customResult = np.around(model.predict(custom), 1)
print('Grade 25 :')
print('expected result : 30.4')
print('predicted result :', customResult[0,0])
print('Grade 30 :')
print('expected result : 42.1')
print('predicted result :', customResult[1,0])

testing custom data for 28 days...
Grade 25 :
expected result : 30.4
predicted result : 29.3
Grade 30 :
expected result : 42.1
predicted result : 31.2


In [ ]:
print('\nmodel trained')
_, accuracy_train = model.evaluate(xr, yr)
print('\nAccuracy on train data: %.2f' % (accuracy_test))

_, accuracy_validation = model.evaluate(xv, yv)
print('\nAccuracy on validation data: %.2f' % (accuracy_test))

_, accuracy_test = model.evaluate(xt, yt)
print('\nAccuracy on test data: %.2f' % (accuracy_test))


model trained
25/25 [==============================] - 0s 2ms/step - loss: 15.2993 - mean_absolute_percentage_error: 10.5712

Accuracy on train data: 18.52
6/6 [==============================] - 0s 2ms/step - loss: 36.6784 - mean_absolute_percentage_error: 17.2929

Accuracy on validation data: 18.52
6/6 [==============================] - 0s 2ms/step - loss: 40.5713 - mean_absolute_percentage_error: 18.5242

Accuracy on test data: 18.52


In [ ]:
result = np.concatenate([xt, yt, test_predictions], axis=1)
data.saveData(result, 'results/' + OUTPUT_TITLE +'.xlsx')

saved data to excel file 
path: results/tals4_12_7days.xlsx


In [ ]:
train_predictions = np.around(model.predict(xr), 1)
result = np.concatenate([xr, yr, train_predictions], axis=1)
data.saveData(result, 'results/' + OUTPUT_TITLE +'_train.xlsx')

validation_predictions = np.around(model.predict(xv), 1)
result = np.concatenate([xv, yv, validation_predictions], axis=1)
data.saveData(result, 'results/' + OUTPUT_TITLE +'_validation.xlsx')


saved data to excel file 
path: results/tals4_12_7days_train.xlsx
saved data to excel file 
path: results/tals4_12_7days_validation.xlsx


In [ ]:
print('\nsaving model weights ...')
output_dir = os.path.join(os.getcwd(), "saved_wights")
model.save_weights(filepath=os.path.join(output_dir, OUTPUT_TITLE + ".h5"))
print('model weights saved')


saving model weights ...
model weights saved


**Trained model with saved weights:**


*   all_dataG_40_20_7days
*   tals4_12_7days
